### TODO per Valentina

- go through the files she sent on Teams
- think once again whats how to improve actions identification - still does mistakes
- Make possible to query two files at once
- get some information on where the fuck does she want to publish?


In [2]:
from llm import load_models

from dotenv import load_dotenv
from questioning import load_qdf, ask_qdfs, pdfs_to_documents, documents_to_vector_store, query_action_detail, query_action_SMART, query_action_list, ask_RAG, filter_action_pages

from logging_config import logging_config
logger = logging_config(f'GPT@JRC','logger.log')
import pandas as pd
from tqdm import tqdm
from prompts import PROMPTS
from pydantic import BaseModel, Field
from typing import List, Literal, Optional

config = {
    'Torino':{
        'pdfs': [
            {'file':'Torino.pdf','action_page_limit': (143, 191)}
        ],
    },
    'Zaragoza':{
        'pdfs': [
            {'file':'Zaragoza.pdf','action_page_limit': (0, 200)}
        ],
    },
    'Bologna':{
        'pdfs': [
            {'file':'Bologna.pdf','action_page_limit': (128, 194)}
        ],
    },
    'Izmir':{
        'pdfs': [
            {'file':'Izmir.pdf','action_page_limit': (58, 142)}
        ],
    },
    'Lisbon':{
        'pdfs': [
            {'file':'Lisbon_1.pdf','action_page_limit': (0, 200)},
            {'file':'Lisbon_2.pdf','action_page_limit': (0, 200)}
        ],
    },
    'Liepaja':{
        'pdfs': [
            {'file':'Liepaja_1.pdf','action_page_limit': (60, 100)},
            #{'file':'Liepaja_2.pdf','action_page_limit': (0, 200)} # seem to be old version
        ],
    },
    'Valencia':{
        'pdfs': [
            {'file':'Valencia_1.pdf','action_page_limit': (0, 200)},
            {'file':'Valencia_2.pdf','action_page_limit': (0, 200)}
        ],
    },
    'Porto':{
        'pdfs': [
            {'file':'Porto_1.pdf','action_page_limit': (0, 200)},
            {'file':'Porto_2.pdf','action_page_limit': (0, 200)}
        ],
    },
    'Lappeenranta':{
        'pdfs': [
            {'file':'Lappeenranta.pdf','action_page_limit': (0, 200)}
            
        ],
    },
    'Firenze':{
        'pdfs': [
            {'file':'Firenze.pdf','action_page_limit': (45, 87)},
        ],
    },
    'Ioannina':{
        'pdfs': [
            {'file':'Ioannina.pdf','action_page_limit': (75, 125)}
        ]
    },
    'Kalamata':{
        'pdfs': [
            {'file':'Kalamata.pdf','action_page_limit': (78, 122)}
        ]
    },
    'Tampere':{
        'pdfs': [
            {'file':'Tampere.pdf','action_page_limit': (28, 62)}
        ],
    },
    'Espoo':{
        'pdfs': [
            {'file':'Espoo.pdf','action_page_limit': (30, 88)}
        ],
    },
    'Guimaraes':{
        'pdfs': [
            {'file':'Guimaraes.pdf','action_page_limit': (226, 282)}
        ]
    },
    'Lahti':{
        'pdfs': [
            {'file':'Lahti.pdf','action_page_limit': (12, 35)}
        ],
    },
    'Kozani':{
        'pdfs': [
            {'file':'Kozani.pdf','action_page_limit': (250, 443)}
        ],
    },
    'Pecs':{
        'pdfs': [
            {'file':'Pecs.pdf','action_page_limit': (30, 78)}
        ],
    },
    'Sevilla':{
        'pdfs': [
            {'file':'Sevilla.pdf','action_page_limit': (190, 300)}
        ],
    },
    'Turku':{
        'pdfs': [
            {'file':'Turku.pdf','action_page_limit': (10, 55)}
        ],
    },
    'Leuven':{
        'pdfs': [
            {'file':'Leuven.pdf','action_page_limit': (17, 51)}
        ]
    }
}

load_dotenv()
llm, emb = load_models(service = 'DeepInfra')
qdf = load_qdf() # questions from JSON
pdf_prefix='pdf_input/'

In [3]:
def pdf_to_excel(city, excel_output, llm=llm, emb=emb, pdf_prefix='pdf_input/',config=config):
    files = [f'{pdf_prefix}{pdf["file"]}' for pdf in config[city]['pdfs']]

    logger.info(f'Starting PDFs of {city}')    
    logger.debug('Reading the documents..')
    documents, splits = pdfs_to_documents(files)
    
    logger.debug('Converting to vector store ...')
    vector_store = documents_to_vector_store(documents, emb)

    with pd.ExcelWriter(excel_output) as xlsx:

        qs_responses = ask_qdfs(
            qdf = qdf,
            llm = llm, 
            vector_store = vector_store,
            city = city,
            logger=logger
        )

        qs_responses.to_excel(xlsx,sheet_name='standard_qs')

        logger.debug('Barriers...')

        class Barrier(BaseModel):
            title: str
            brief_description: str
            page_reference: str
            explanation: str
            category: List[Literal["Leadership", "Financial", "Regulatory", "Operational", "Organisational", "Partnerships", "Social", "Environmental", "Safety and Security", "Other"]]
        
        class BarrierList(BaseModel):
            barriers: List[Barrier]


        response = ask_RAG(
            embed_query = '''Identify text segments that describe obstacles, limitations, or difficulties in implementing the SECAP or achieving emission reduction targets. 
            Focus on segments describing: "barrier", "challenge", "weakness", "lack of", "difficulty/difficulties", "insufficient/insufficiency", "inadequate/inadequacy",
            "failure", "underestimation", "burden" when they specifically relate to plan implementation or target achievement. 
            
            Exclude mentions of barriers that only apply to specific individual actions.''',
            vector_store = vector_store,
            response_model=BarrierList,
            llm = llm,
            logger = logger,
            template_key = 'barriers_prompt',
            template_kwargs={},
            city = city,
            k=50
        )
        bars = pd.DataFrame([b for b in response['barriers']])
        bars.to_excel(xlsx, sheet_name = 'barriers')

        logger.debug('Participatory processes...')

        class Stakeholder(BaseModel):
            stakeholder: str
            role: str

        class ParticipatoryProcess(BaseModel):
            title: str
            brief_description: str = Field(description="Description in English")
            page_reference: str
            explanation: str
            relevant_stakeholders: List[Stakeholder]
            implementation_barriers: Optional[List[str]] = []
            barriers_solution: Optional[List[str]] = []
            citizen_contribution_clearly_defined: bool

        class ParticipatoryProcesses(BaseModel):
            participatory_processes: List[ParticipatoryProcess]


        response = ask_RAG(
            embed_query='''
            1. Identify sections that describe formal participatory processes using terms like: "consultation", "workshop", "public hearing", "stakeholder meeting", "engagement process", "participatory planning", "co-creation"
            2. Identify text segments describing specific mechanisms for citizen input such as: "feedback", "survey", "questionnaire", "public comment", "citizen proposal", "voting", "participatory budgeting"
            3. Identify descriptions of stakeholder groups and their roles using terms like: "working group", "advisory board", "steering committee", "citizen panel", "focus group", "community representative"
            4. Identify segments discussing challenges or limitations in public participation using terms like: "barrier", "challenge", "limitation", "difficulty", "low participation", "engagement gap"            ''',
            template_key='participatory_processes',
            template_kwargs={},
            response_model=ParticipatoryProcesses,
            vector_store=vector_store,
            llm=llm,
            logger=logger,
            city=city,
            k=50
        )

        participatory_processes = pd.DataFrame([pp for pp in response['participatory_processes']])
        participatory_processes.to_excel(xlsx, sheet_name='participatory_processes')

        logger.info(f'Action list...')

        action_list = query_action_list(
            llm=llm, 
            documents=filter_action_pages(splits, config[city]),
            logger=logger
        )

        logger.info(f'Action details...')
        ads = [query_action_detail(
                action=action, 
                llm=llm, 
                vector_store=vector_store, 
                logger=logger,
                city = city
            ) 
            for action in tqdm(action_list.to_dict(orient='records'),desc='Action details')]
        action_details = pd.json_normalize([ad for ad in ads if ad is not None])

        action_details.to_excel(xlsx, sheet_name='Actions')
        logger.info(f'Action SMARTs...')

        smarts = [query_action_SMART(
                action=action,
                llm=llm,
                vector_store=vector_store, 
                logger=logger,
                city = city
            ) for action in tqdm(action_list.to_dict(orient='records'),desc='Action SMART')]
        action_SMART = pd.json_normalize([sm for sm in smarts if sm is not None])

        action_SMART.to_excel(xlsx, sheet_name='Actions_SMART')

for city in ['Firenze']:#list(config.keys()):
    pdf_to_excel(
        city, 
        f'output/{city}.xlsx')

2025-01-28 15:14:21,842 - GPT@JRC - INFO - Starting PDFs of Firenze


Questions: 100%|██████████| 58/58 [13:14<00:00, 13.70s/it]


2025-01-28 15:28:09,981 - GPT@JRC - INFO - Action list...
2025-01-28 15:33:00,134 - GPT@JRC - INFO - Action details...


Action details: 100%|██████████| 35/35 [31:45<00:00, 54.44s/it]

2025-01-28 16:04:45,715 - GPT@JRC - INFO - Action SMARTs...



Action SMART: 100%|██████████| 35/35 [07:27<00:00, 12.79s/it]


In [2]:
city = 'Firenze'
files = [f'{pdf_prefix}{pdf["file"]}' for pdf in config[city]['pdfs']]

documents, splits = pdfs_to_documents(files)

vector_store = documents_to_vector_store(documents, emb)

In [ ]:
class Barrier(BaseModel):
    title: str
    brief_description: str
    page_reference: str
    explanation: str
    category: List[Literal["Leadership", "Financial", "Regulatory", "Operational", "Organisational", "Partnerships", "Social", "Environmental", "Safety and Security", "Other"]]

class BarrierList(BaseModel):
    barriers: List[Barrier]


response = ask_RAG(
    embed_query = '''Identify text segments that describe obstacles, limitations, or difficulties in implementing the SECAP or achieving emission reduction targets. 
    Focus on segments describing: "barrier", "challenge", "weakness", "lack of", "difficulty/difficulties", "insufficient/insufficiency", "inadequate/inadequacy",
    "failure", "underestimation", "burden" when they specifically relate to plan implementation or target achievement. 
    
    Exclude mentions of barriers that only apply to specific individual actions.''',
    vector_store = vector_store,
    response_model=BarrierList,
    llm = llm,
    logger = logger,
    template_key = 'barriers_prompt',
    template_kwargs={},
    city = city,
    k=50
)
bars = pd.DataFrame([b for b in response['barriers']])
bars

In [ ]:
response